In [288]:
import numpy as np
import pandas as pd
import sys

In [289]:
trainpd=pd.read_csv("Dataset/mnist/train.csv",header=None,sep=',')
testpd=pd.read_csv("Dataset/mnist/test.csv",header=None,sep=',')

In [290]:
class SVM:
    
    def __init__(self,x,y,normalize_x=True):
        self.x=np.copy(x).astype(float)
        self.y=np.copy(y)
        [instances,dimensions]=x.shape
        self.dimensions=dimensions
        self.instances=instances
        self.normalize_param=np.ones((1,dimensions))
        self.normalize_x=normalize_x
        label=np.unique(y).shape[0]
        self.label=label
        self.weight=np.zeros((self.label,self.label,dimensions))
        self.bias=np.zeros((self.label,self.label))
        self.epoch=np.zeros((self.label,self.label))
        self.train_steps=np.zeros((self.label,self.label))

        if(normalize_x==True):
            self.normalize_param=self.getNormParam()
            self.x=self.normalize(self.x)
        
    def getNormParam(self):
        max_arr=np.amax(self.x,axis=0)
        bool_0=(max_arr==0)
        max_arr+=bool_0.astype(int)
        return max_arr[np.newaxis,:]
    
    def normalize(self,x):
        return (x/self.normalize_param)
    
    def resetParam(self,i,j):
        self.weight[i,j,:]*=0
        self.bias[i,j]=0
        self.epoch[i,j]=0
        self.train_steps[i,j]=0
        return
    
    def evaluate(self,x,y,weight,bias):
        return (np.matmul(x,weight)+bias)*y
    
    def getLoss(self,x,y,lamda,C,weight,bias):
        loss=np.matmul(np.transpose(weight),weight)*lamda
        evaluate=1-self.evaluate(x=x,y=y,weight=weight,bias=bias)
        mask=(evaluate>0).astype(int)
        evaluate*=mask
        loss+=C*np.sum(evaluate)
        return loss
    
    def getWeight(self,i,j):
        return self.weight[i,j,:][:,np.newaxis]
    
    def train1Classier(self,weight,bias,train_steps,epoch,x=None,y=None,lamda=1,C=1,max_steps=100000,batch_mode=True,batch_size=100,epsilon=0.0001,log_every=2000):

        if(x is None):
            x=self.x
        if(y is None):
            y=self.y
            
        xy=-(x*y)
        [instances,dim]=x.shape
        [instances,ydim]=y.shape
        
        if(batch_mode==False):
            batch_size=instances
        
        prev_loss=self.getLoss(x=x,y=y,lamda=lamda,C=C,weight=weight,bias=bias)
            
        Trained=False    
        cur_steps=0
        while(Trained==False and cur_steps<max_steps):
            cur_instance=0
            epoch+=1
            
            while(cur_instance<instances and cur_steps<max_steps):
                train_steps+=1
                cur_steps+=1
                eta=1.0/(lamda*cur_steps)
                print(eta)
                up_lim=min(instances,cur_instance+batch_size)
                batch_x=x[cur_instance:up_lim,0:dim]
                batch_y=y[cur_instance:up_lim,0:ydim]
                batch_xy=xy[cur_instance:up_lim,0:dim]
                examples=up_lim-cur_instance
                cur_instance=up_lim
                evaluate=self.evaluate(batch_x,batch_y,weight,bias)
                mask=((1-evaluate)>0).astype(int)
                weight=(1-eta*lamda)*weight + (eta*C*(np.sum(batch_xy*mask,axis=0)[:,np.newaxis]))/examples
                bias=bias+((np.sum(mask*batch_y))*C*eta)/examples
                
                if(train_steps%log_every==0):
                    print("Batch Loss at Steps=",train_steps," is ",self.getLoss(x,y,lamda,C,weight,bias))
            
            cur_loss=self.getLoss(x,y,lamda,C,weight,bias)
            if(abs(cur_loss-prev_loss)<epsilon):
                Trained=True
                
            prev_loss=cur_loss

        return (weight,bias,epoch,train_steps)
    
    def trainIJclassifier(self,i,j,reset=False):
        
        if(reset==True):
            self.resetParam(i,j)
        
        print("********************************************************")
        print(i," ",j," ","Classifier")
        
        aux_arr=(self.y==i).astype(int)+(self.y==j).astype(int)

        newx=self.x[aux_arr[:,0]>0]
        newy=self.y[aux_arr[:,0]>0]
        newy=np.copy(newy)
        newy=(newy==i).astype(int)

        (weight,bias,epoch,train_steps)=self.train1Classier(weight=self.getWeight(i,j),
                                          bias=self.bias[i,j],
                                          x=newx,
                                          y=newy,
                                          train_steps=self.train_steps[i,j],
                                          epoch=self.epoch[i,j])
        
        self.weight[i,j,:]=weight[:,0]
        self.bias[i,j]=bias
        self.epoch[i,j]=epoch
        self.train_steps[i,j]=train_steps
        
    def train(self):
        for i in range(self.label):
            for j in range(i+1,self.label,1):
                self.trainIJclassifier(i,j)
#                 return
        
        

            

In [291]:
def parseDataframe(dataframe):
    values=dataframe.values
    [instances,dim]=values.shape
    x=values[:,0:dim-1]
    y=values[:,dim-1:dim]
    return (x,y)

In [292]:
(x_train,y_train)=parseDataframe(trainpd)

In [293]:
print(x_train.shape)
print(y_train.shape)

(20000, 784)
(20000, 1)


In [294]:
SVM_instance=SVM(x_train,y_train)

In [295]:
SVM_instance.train()

********************************************************
0   1   Classifier
1.0
0.5
0.3333333333333333
0.25
0.2
0.16666666666666666
0.14285714285714285
0.125
0.1111111111111111
0.1
0.09090909090909091
0.08333333333333333
0.07692307692307693
0.07142857142857142
0.06666666666666667
0.0625
0.058823529411764705
0.05555555555555555
0.05263157894736842
0.05
0.047619047619047616
0.045454545454545456
0.043478260869565216
0.041666666666666664
0.04
0.038461538461538464
0.037037037037037035
0.03571428571428571
0.034482758620689655
0.03333333333333333
0.03225806451612903
0.03125
0.030303030303030304
0.029411764705882353
0.02857142857142857
0.027777777777777776
0.02702702702702703
0.02631578947368421
0.02564102564102564
0.025
0.024390243902439025
0.023809523809523808
0.023255813953488372
0.022727272727272728
0.022222222222222223
0.021739130434782608
0.02127659574468085
0.020833333333333332
0.02040816326530612
0.02
0.0196078431372549
0.019230769230769232
0.018867924528301886
0.018518518518518517
0.0

0.0024390243902439024
0.0024330900243309003
0.0024271844660194173
0.002421307506053269
0.0024154589371980675
0.0024096385542168677
0.002403846153846154
0.002398081534772182
0.0023923444976076554
0.002386634844868735
0.002380952380952381
0.0023752969121140144
0.002369668246445498
0.002364066193853428
0.0023584905660377358
0.002352941176470588
0.002347417840375587
0.00234192037470726
0.002336448598130841
0.002331002331002331
0.002325581395348837
0.002320185614849188
0.0023148148148148147
0.0023094688221709007
0.002304147465437788
0.0022988505747126436
0.0022935779816513763
0.002288329519450801
0.00228310502283105
0.002277904328018223
0.0022727272727272726
0.0022675736961451248
0.0022624434389140274
0.002257336343115124
0.0022522522522522522
0.0022471910112359553
0.002242152466367713
0.0022371364653243847
0.002232142857142857
0.0022271714922048997
0.0022222222222222222
0.0022172949002217295
0.0022123893805309734
0.002207505518763797
0.0022026431718061676
0.002197802197802198
0.00219298245

0.001182033096926714
0.0011806375442739079
0.0011792452830188679
0.001177856301531213
0.001176470588235294
0.0011750881316098707
0.0011737089201877935
0.0011723329425556857
0.00117096018735363
0.0011695906432748538
0.0011682242990654205
0.0011668611435239206
0.0011655011655011655
0.0011641443538998836
0.0011627906976744186
0.0011614401858304297
0.001160092807424594
0.0011587485515643105
0.0011574074074074073
0.0011560693641618498
0.0011547344110854503
0.0011534025374855825
0.001152073732718894
0.0011507479861910242
0.0011494252873563218
0.001148105625717566
0.0011467889908256881
0.001145475372279496
0.0011441647597254005
0.001142857142857143
0.001141552511415525
0.0011402508551881414
0.0011389521640091116
0.0011376564277588168
0.0011363636363636363
0.0011350737797956867
0.0011337868480725624
0.0011325028312570782
0.0011312217194570137
0.0011299435028248588
0.001128668171557562
0.0011273957158962795
0.0011261261261261261
0.0011248593925759281
0.0011235955056179776
0.001122334455667789
0

0.000758150113722517
0.0007575757575757576
0.000757002271006813
0.0007564296520423601
0.0007558578987150416
0.0007552870090634441
0.0007547169811320754
0.0007541478129713424
0.0007535795026375283
0.0007530120481927711
0.0007524454477050414
0.0007518796992481203
0.0007513148009015778
0.0007507507507507507
0.0007501875468867217
0.0007496251874062968
0.000749063670411985
0.0007485029940119761
0.0007479431563201197
0.0007473841554559044
0.0007468259895444362
0.0007462686567164179
0.0007457121551081282
0.0007451564828614009
0.0007446016381236039
0.000744047619047619
0.0007434944237918215
0.0007429420505200594
0.0007423904974016332
0.000741839762611276
0.0007412898443291327
0.0007407407407407407
0.0007401924500370096
0.0007396449704142012
0.0007390983000739098
0.0007385524372230429
0.0007380073800738007
0.0007374631268436578
0.0007369196757553427
0.0007363770250368188
0.0007358351729212656
0.0007352941176470588
0.0007347538574577516
0.0007342143906020558
0.0007336757153338225
0.0007331378299

0.000526592943654555
0.0005263157894736842
0.0005260389268805891
0.0005257623554153522
0.0005254860746190226
0.0005252100840336134
0.0005249343832020997
0.0005246589716684155
0.0005243838489774515
0.0005241090146750524
0.0005238344683080147
0.0005235602094240838
0.0005232862375719519
0.0005230125523012552
0.0005227391531625719
0.0005224660397074191
0.0005221932114882506
0.0005219206680584551
0.0005216484089723526
0.0005213764337851929
0.0005211047420531526
0.0005208333333333333
0.0005205622071837585
0.0005202913631633715
0.0005200208008320333
0.0005197505197505198
0.0005194805194805195
0.0005192107995846313
0.0005189413596263622
0.0005186721991701245
0.0005184033177812338
0.0005181347150259067
0.0005178663904712584
0.0005175983436853002
0.0005173305742369374
0.000517063081695967
0.0005167958656330749
0.0005165289256198347
0.0005162622612287042
0.0005159958720330237
0.0005157297576070139
0.0005154639175257732
0.0005151983513652757
0.0005149330587023687
0.000514668039114771
0.00051440329

...

5.048975058063213e-05
5.0487201494421164e-05
5.0484652665589664e-05
5.0482104094098645e-05
5.047955577990914e-05
5.0477007722982184e-05
5.047445992327882e-05
5.047191238076011e-05
5.04693650953871e-05
5.046681806712087e-05
5.0464271295922484e-05
5.0461724781753044e-05
5.045917852457362e-05
5.0456632524345326e-05
5.0454086781029264e-05
5.045154129458655e-05
5.044899606497831e-05
5.044645109216567e-05
5.0443906376109764e-05
5.044136191677175e-05
5.043881771411278e-05
5.043627376809401e-05
5.043373007867662e-05
5.0431186645821776e-05
5.042864346949067e-05
5.04261005496445e-05
5.042355788624445e-05
5.0421015479251755e-05
5.041847332862761e-05
5.041593143433325e-05
5.0413389796329905e-05
5.041084841457882e-05
5.0408307289041234e-05
5.040576641967841e-05
5.040322580645161e-05
5.040068544932211e-05
5.0398145348251183e-05
5.039560550320012e-05
5.0393065914130216e-05
5.039052658100277e-05
5.03879875037791e-05
5.0385448682420515e-05
5.0382910116888354e-05
5.038037180714394e-05
5.0377833753148616

4.918355301987015e-05
4.9181134116952735e-05
4.91787154519524e-05
4.917629702483403e-05
4.917387883556255e-05
4.917146088410287e-05
4.9169043170419906e-05
4.9166625694478586e-05
4.9164208456243856e-05
4.916179145568064e-05
4.915937469275391e-05
4.91569581674286e-05
4.915454187966968e-05
4.915212582944212e-05
4.91497100167109e-05
4.9147294441441e-05
4.914487910359741e-05
4.9142464003145115e-05
4.914004914004914e-05
4.9137634514274483e-05
4.9135220125786166e-05
4.913280597454921e-05
4.913039206052864e-05
4.9127978383689514e-05
4.912556494399686e-05
4.912315174141573e-05
4.912073877591119e-05
4.9118326047448304e-05
4.911591355599214e-05
4.911350130150779e-05
4.9111089283960316e-05
4.9108677503314834e-05
4.910626595953644e-05
4.910385465259023e-05
4.910144358244132e-05
4.909903274905484e-05
4.909662215239592e-05
4.909421179242967e-05
4.909180166912126e-05
4.9089391782435816e-05
4.908698213233851e-05
4.908457271879448e-05
4.908216354176892e-05
4.907975460122699e-05
4.9077345897133884e-05
4.

4.803996925441968e-05
4.803766152663688e-05
4.8035354020559135e-05
4.8033046736154476e-05
4.803073967339097e-05
4.802843283223668e-05
4.8026126212659685e-05
4.802381981462805e-05
4.8021513638109875e-05
4.801920768307323e-05
4.801690194948622e-05
4.8014596437316946e-05
4.801229114653351e-05
4.800998607710404e-05
4.800768122899664e-05
4.8005376602179446e-05
4.8003072196620584e-05
4.8000768012288194e-05
4.799846404915043e-05
4.7996160307175425e-05
4.799385678633135e-05
4.799155348658636e-05
4.7989250407908626e-05
4.798694755026633e-05
4.798464491362764e-05
4.798234249796075e-05
4.798004030323385e-05
4.797773832941515e-05
4.797543657647284e-05
4.797313504437515e-05
4.797083373309028e-05
4.796853264258646e-05
4.7966231772831926e-05
4.7963931123794903e-05
4.796163069544364e-05
4.795933048774639e-05
4.79570305006714e-05
4.795473073418693e-05
4.795243118826125e-05
4.795013186286262e-05
4.794783275795934e-05
4.794553387351968e-05
4.794323520951194e-05
4.794093676590441e-05
4.7938638542665386e-0

4.690651531497725e-05
4.6904315196998124e-05
4.690211528539937e-05
4.689991558015196e-05
4.689771608122684e-05
4.689551678859501e-05
4.6893317702227434e-05
4.6891118822095096e-05
4.6888920148168986e-05
4.6886721680420107e-05
4.688452341881945e-05
4.688232536333802e-05
4.6880127513946835e-05
4.687792987061691e-05
4.687573243331927e-05
4.6873535202024934e-05
4.687133817670495e-05
4.6869141357330336e-05
4.6866944743872144e-05
4.686474833630143e-05
4.686255213458925e-05
4.6860356138706655e-05
4.6858160348624716e-05
4.6855964764314496e-05
4.685376938574708e-05
4.685157421289355e-05
4.684937924572499e-05
4.68471844842125e-05
4.6844989928327164e-05
4.6842795578040096e-05
4.68406014333224e-05
4.68384074941452e-05
4.68362137604796e-05
4.683402023229674e-05
4.683182690956774e-05
4.682963379226374e-05
4.682744088035589e-05
4.682524817381532e-05
4.6823055672613196e-05
4.6820863376720664e-05
4.68186712861089e-05
4.6816479400749066e-05
4.681428772061233e-05
4.681209624566988e-05
4.68099049758929e-05

KeyboardInterrupt: 